In [42]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

today = datetime.today().strftime('%Y-%m-%d')
start_date = '2019-01-01'

eth_df = yf.download('BTC-USD',start_date, today)

eth_df.tail()

In [43]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2010-01-01'

eth_df = yf.download('BTC-USD',start_date, today)

eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-23,"$42,364.38","$42,893.51","$41,877.51","$42,892.96","$42,892.96",25242943069
2022-03-24,"$42,886.65","$44,131.86","$42,726.16","$43,960.93","$43,960.93",31042992291
2022-03-25,"$43,964.55","$44,999.49","$43,706.29","$44,348.73","$44,348.73",30574413034
2022-03-26,"$44,349.86","$44,736.00","$44,166.27","$44,500.83","$44,500.83",16950455995
2022-03-27,"$44,505.36","$46,827.55","$44,437.29","$46,820.49","$46,820.49",28160889722


In [44]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2749 entries, 2014-09-17 to 2022-03-27
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2749 non-null   float64
 1   High       2749 non-null   float64
 2   Low        2749 non-null   float64
 3   Close      2749 non-null   float64
 4   Adj Close  2749 non-null   float64
 5   Volume     2749 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 150.3 KB


In [45]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [46]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [47]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [48]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds", 
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [49]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [50]:
from fbprophet.plot import add_changepoints_to_plot

m = Prophet(
    seasonality_mode="multiplicative",
    changepoint_prior_scale=0.005,
    yearly_seasonality = True
)
m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
m= m.add_seasonality(name="quarterly", period=92.25, fourier_order=10)
m.fit(df)
#fig = m.plot(forecast)
#a = add_changepoints_to_plot(fig.gca(), m, forecast)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -69.9675


    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       5083.71     0.0030936           671           1           1      125   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     106       5089.92    0.00503314       1072.11   5.663e-06       0.001      172  LS failed, Hessian reset 
     199       5096.48   0.000911755       810.579           1           1      289   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     232       5112.03    0.00495029        687.65   6.708e-06       0.001      362  LS failed, Hessian reset 
     299       5122.78   3.08399e-05       701.835       1.209      0.1209      453   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     318       5122.85   2.82016e-05       774.587   3.076e-08       0.001      513  LS failed, Hessian reset 
     399       5123.04   2.18883e-06  

In [51]:
future = m.make_future_dataframe(periods = 365)


In [52]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
3109,2023-03-23,"$77,573.23","$70,428.74","$84,139.05"
3110,2023-03-24,"$77,784.84","$70,528.24","$84,444.30"
3111,2023-03-25,"$77,737.35","$71,256.15","$84,425.37"
3112,2023-03-26,"$78,095.79","$71,125.67","$85,073.79"
3113,2023-03-27,"$80,573.76","$73,828.84","$87,522.03"


In [53]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

73207.27053876138

In [54]:
plot_plotly(m, forecast)

In [55]:
plot_components_plotly(m, forecast)